Problem Statement:
A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 
Approach - A Random Forest can be built with target variable Sales (we will first convert it in categorical variable) & all other variable will be independent in the analysis.  


In [1]:
#load data set
import pandas as pd
company_data=pd.read_csv("Company_Data.csv")
company_data

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


## EDA Perform

In [2]:
company_data['ShelveLoc'].replace({'Bad':0,'Medium':1,'Good':2},inplace=True)
company_data['Urban'].replace({'No':0,'Yes':1},inplace=True)
company_data['US'].replace({'No':0,'Yes':1},inplace=True)

In [3]:
company_data

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,2,65,10,1,1
2,10.06,113,35,10,269,80,1,59,12,1,1
3,7.40,117,100,4,466,97,1,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,2,33,14,1,1
396,6.14,139,23,3,37,120,1,55,11,0,1
397,7.41,162,26,12,368,159,1,40,18,1,1
398,5.94,100,79,7,284,95,0,50,12,1,1


In [4]:
data=company_data.copy()

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    int64  
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    int64  
 10  US           400 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 34.5 KB


In [6]:
# Check for outliers
from sklearn.ensemble import IsolationForest

In [7]:
clf=IsolationForest(contamination=0.01,random_state=10)
clf.fit(data)

IsolationForest(contamination=0.01, random_state=10)

In [8]:
data['scores']=clf.decision_function(data)

In [9]:
data['anomaly']=clf.predict(data.iloc[:,0:11])

In [10]:
data

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,scores,anomaly
0,9.50,138,73,11,276,120,0,42,17,1,1,0.119919,1
1,11.22,111,48,16,260,83,2,65,10,1,1,0.091476,1
2,10.06,113,35,10,269,80,1,59,12,1,1,0.148863,1
3,7.40,117,100,4,466,97,1,55,14,1,1,0.159319,1
4,4.15,141,64,3,340,128,0,38,13,1,0,0.120977,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,2,33,14,1,1,0.082170,1
396,6.14,139,23,3,37,120,1,55,11,0,1,0.068608,1
397,7.41,162,26,12,368,159,1,40,18,1,1,0.063488,1
398,5.94,100,79,7,284,95,0,50,12,1,1,0.132051,1


In [11]:
data[data['anomaly']==-1]

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,scores,anomaly
75,8.55,88,111,23,480,92,0,36,16,0,1,-0.021892,-1
165,0.37,147,58,7,100,191,0,27,15,1,1,-0.009228,-1
174,0.00,139,24,0,358,185,1,79,15,0,0,-0.006591,-1
367,14.37,95,106,0,256,53,2,52,17,1,0,-0.018025,-1


In [12]:
data=data.drop(data.index[[75,165,174,367]],axis=0).reset_index()

In [13]:
data

,index,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,scores,anomaly
0,0,9.50,138,73,11,276,120,0,42,17,1,1,0.119919,1
1,1,11.22,111,48,16,260,83,2,65,10,1,1,0.091476,1
2,2,10.06,113,35,10,269,80,1,59,12,1,1,0.148863,1
3,3,7.40,117,100,4,466,97,1,55,14,1,1,0.159319,1
4,4,4.15,141,64,3,340,128,0,38,13,1,0,0.120977,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,395,12.57,138,108,17,203,128,2,33,14,1,1,0.082170,1
392,396,6.14,139,23,3,37,120,1,55,11,0,1,0.068608,1
393,397,7.41,162,26,12,368,159,1,40,18,1,1,0.063488,1
394,398,5.94,100,79,7,284,95,0,50,12,1,1,0.132051,1


In [14]:
data=data.drop(['index'],axis=1)

In [15]:
data

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,scores,anomaly
0,9.50,138,73,11,276,120,0,42,17,1,1,0.119919,1
1,11.22,111,48,16,260,83,2,65,10,1,1,0.091476,1
2,10.06,113,35,10,269,80,1,59,12,1,1,0.148863,1
3,7.40,117,100,4,466,97,1,55,14,1,1,0.159319,1
4,4.15,141,64,3,340,128,0,38,13,1,0,0.120977,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,12.57,138,108,17,203,128,2,33,14,1,1,0.082170,1
392,6.14,139,23,3,37,120,1,55,11,0,1,0.068608,1
393,7.41,162,26,12,368,159,1,40,18,1,1,0.063488,1
394,5.94,100,79,7,284,95,0,50,12,1,1,0.132051,1


In [16]:
X=data.iloc[:,1:9]
Y=data['Sales']

## Random Forest as Regressor

In [17]:
#model validation technique(train test split)
from sklearn.model_selection import train_test_split

In [18]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=55)

In [19]:
#Build a model
from sklearn.ensemble import RandomForestRegressor

In [20]:
model=RandomForestRegressor(n_estimators=96,max_features=3,random_state=5)
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.6993597483744871

In [21]:
#model validation technique(K-Fold)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [22]:
kfold=KFold(n_splits=7)
results=cross_val_score(model,X,Y,cv=kfold)
results.mean()

0.6580853329317736

In [23]:
#model validation technique(LooCV)
#from sklearn.model_selection import LeaveOneOut

In [24]:
#loocv=LeaveOneOut()
#result=cross_val_score(model,X,Y,cv=loocv)
#result.mean()*100


In [25]:
#feature engeeniaring
model.feature_importances_*100

array([10.22048323,  7.80868407,  9.73031585,  6.34381053, 26.07432248,
       22.07377039, 13.10002103,  4.64859243])

## Random Forest as Classifier

In [26]:
#now make targate variable as categorical
y=[0 if b<7.5 else 1 for b in Y]
y=pd.DataFrame(y)
y.value_counts()  # check balance data

0    199
1    197
dtype: int64

In [27]:
#model validation technique(train test split)
X_TRAIN,X_TEST,Y_TRAIN,Y_TEST=train_test_split(X,y,test_size=0.2,random_state=55)

In [28]:
#Build a model
from sklearn.ensemble import RandomForestClassifier

In [29]:
model2=RandomForestClassifier(n_estimators=114,max_features=3,random_state=55)
model2.fit(X_TRAIN,Y_TRAIN)
model2.score(X_TEST,Y_TEST)

C:\Users\1234\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.825

In [30]:
Y_PRED=model2.predict(X_TEST)
from sklearn.metrics import classification_report
print(classification_report(Y_TEST,Y_PRED))

              precision    recall  f1-score   support

           0       0.86      0.82      0.84        45
           1       0.78      0.83      0.81        35

    accuracy                           0.82        80
   macro avg       0.82      0.83      0.82        80
weighted avg       0.83      0.82      0.83        80



In [32]:
#model validation technique(k-fold)
k_fold=KFold(n_splits=7)
RESULTS=cross_val_score(model2,X,y,cv=k_fold)
RESULTS.mean()

C:\Users\1234\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\1234\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\1234\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\1234\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was pa

0.8282760472610097